In [3]:

search_url = "http://cnrail.geogv.org/api/v1/match_feature/{station}?locale=zhcn&query-override="
station_url = "http://cnrail.geogv.org/api/v1/station/{station_code}?locale=zhcn&query-override=&requestGeom=true"
station_link_url = "http://cnrail.geogv.org/api/v1/station-link/{station_code}?locale=zhcn&query-override="
rail_url = "http://cnrail.geogv.org/api/v1/rail/{rail_id}?locale=zhcn"

header = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Language": "zh-CN,zh;q=0.9,zh-TW;q=0.8,en-US;q=0.7,en;q=0.6",
    "Cache-Control": "max-age=0",
    "Cookie": "_ga=GA1.2.1859979260.1734605488; _ga_ZNGLP4B8Z0=GS1.2.1734605490.1.0.1734605490.0.0.0; _gid=GA1.2.1767102999.1735281206; _ga_9929PFMX6M=GS1.2.1735281205.4.1.1735282204.0.0.0",
    # "If-None-Match": "W/\"184-E+2KUooxUSOLH8UrTF9oPA\"",
    "Proxy-Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
}



In [4]:
station = dict()
rails = dict()
station_name_wait_search = ["北京"]
station_codes = list()
rail_codes = list()

error_station= list()
error_rail = list()

In [5]:
import requests
import json

while True:
    if len(station_name_wait_search) == 0 and len(station_codes) == 0 and len(rail_codes) == 0:
        break
    print("search status:", len(station_name_wait_search), len(station_codes), len(rail_codes))

    while len(station_name_wait_search):
        source_station = station_name_wait_search[0]
        station_name_wait_search = station_name_wait_search[1:]
        resp = requests.get(search_url.format(station=source_station), headers=header)
        resp = json.loads(resp.text)

        if resp["success"] == False:
            print("search station error: ", source_station)
            continue

        station_codes.extend(resp["data"])


    while len(station_codes):
        code, _type, name = station_codes[0]

        if _type not in ["S", "STATION"]:
            station_codes = station_codes[1:]
            continue
        if code in station:
            station_codes = station_codes[1:]
            continue

        resp = requests.get(station_url.format(station_code=code), headers=header)
        resp = json.loads(resp.text)

        station[code] = resp
        print("new station: ", name, code)

        resp = requests.get(station_link_url.format(station_code=code), headers=header)
        resp = json.loads(resp.text)
        if resp["success"] == False:
            print("search rail error: ", name)
            error_station.append(station_codes[0])
            station_codes = station_codes[1:]
            continue

        rail_codes.extend(resp["data"])
        station_codes = station_codes[1:]

    while len(rail_codes):
        rail_info = rail_codes[0]

        if rail_info["railId"] in rails:
            rail_codes = rail_codes[1:]
            continue

        resp = requests.get(rail_url.format(rail_id=rail_info["railId"]), headers=header)
        resp = json.loads(resp.text)
        if resp["success"] == False:
                print("search rail info error: ", rail_info["railName"])
                error_rail.append(rail_codes[0])
                rail_codes = rail_codes[1:]
                continue
        if resp["data"]["diagram"] is None:
            print("search rail info error: ", rail_info["railName"])
            error_rail.append(rail_codes[0])
            rail_codes = rail_codes[1:]
            continue

        print("new rails: ", rail_info["railName"], rail_info["railId"])
        rails[rail_info["railId"]] = resp["data"]

        for record in resp["data"]["diagram"]["records"]:
            for r in record[3]:
                if r[0] != "S":
                    continue
                _type, code, name = r
                station_codes.append([code, _type, name])
                # station_name_wait_search.append(name)
        rail_codes = rail_codes[1:]
            

search status: 1 0 0
new station:  北京 11710233
new station:  北京丰台 630610
new station:  北京西 3100682
new station:  北京南 5900805
new station:  北京东 11720703
new station:  北京北 6560141
new station:  北京城市副中心 97449582
search rail error:  北京城市副中心
new station:  北京大兴 90044941
new station:  北京朝阳 92111611
search rail error:  北京朝阳
new rails:  京沪线 3360
new rails:  北京直通线 31266
new rails:  京哈线 48317
new rails:  丰沙线 8031
new rails:  京广线 1098
new rails:  丰广线 312419
new rails:  丰双线 313892
new rails:  京石客运专线 5170
new rails:  京九线 2034
search rail info error:  西长线
new rails:  京沪高速线 32148
new rails:  京津城际线 4237
new rails:  京张高速线 717872
new rails:  京雄城际线 675520
search status: 0 700 0
new station:  黄村 19160105
new station:  魏善庄 25720894
new station:  安定 25730027
new station:  万庄 25740054
new station:  广阳 25750511
new station:  落垡 25760192
new station:  豆张庄 25770177
new station:  杨村 25780821
new station:  汉沟镇 25790294
new station:  北仓 25800390
new station:  南仓 25810174
new station:  天津西 5920363
new station:  曹庄 2

In [20]:
open("_stations.json", "w", encoding='').write(json.dumps(station, ensure_ascii=False, indent=4))
open("_rails.json", "w").write(json.dumps(rails, ensure_ascii=False, indent=4))

In [22]:
sset = set()
new_station = dict()

for sid in station:
    st = station[sid]
    if st["country"] != "CN":
        continue
    name = st["localizedName"]
    new_station[name] = st

open("_stations.json", "w", encoding='utf-8').write(json.dumps(new_station, ensure_ascii=False, indent=4))

4063126

In [51]:
tobefilter = [
    'TRAJEKT', # 轮渡 过滤
    'TUNNEL1', # 隧道 过滤
    'WASSERq', # 大桥 过滤
    'WBRÜCKE', # 河 过滤
    'WBRüCKE;lZOLL', # 河 过滤
    'WTUNNEL', # 隧道 过滤
    'WTUNNEL1', # 隧道 过滤
    'STR+GRZq', # 关卡 过滤
    'GRZq',  # 关卡 过滤
    'CONTf',
    'CONTg',
    ]

filter_control_set = [
 'ABZ+lr',
 'ABZlf',
 'ABZlg',
 'ABZlr',
 'ABZrf',
 'ABZrg',
 'KRZo',
 'STR',
 'STRf',
 'STRg',
 'STRlf',
 'STRlg',
 'STRrf',
 'STRrg',
 'ENDEe',
 'KBHFa',
]

rails = json.loads(open("_rails.json", encoding='utf-8').read())
stations = json.loads(open("stations.json", encoding='utf-8').read())
new_rails = dict()
rail_type = set()

for rid in rails:
    rail = rails[rid]
    records = rail["diagram"]["records"]
    valid = False
    name = rail["name"]
    if name in new_rails:
        print("error, repeat name", name)

    if rail["railService"] == "F":
        continue

    rail_type.add(rail["railType"])
    continue
    if rail["railType"] == "FERRY":
        continue

    for record in records:
        for x in record[0].split("|"):
            if x in filter_control_set:
                valid = True
    if not valid:
        continue
    print(name)

    diagram = list()
    now = None
    for record in records:
        filter = False
        for x in record[0].split("|"):
            if x in tobefilter:
                filter = True
        if filter:
            continue
        station = record[3]

        if len(station) == 0:
            continue
        if len(station) > 1:
            print("error station > 1", record)
        station = station[0]
        if station[0] != 'S':
            print("error, station != S", record)
            continue
        s_name = station[2]
        if s_name not in stations:
            print("error, not find station", record)

        if now is None:
            now = s_name
        else:
            diagram.append([now, s_name])
        now = s_name
    rail["diagram"] = diagram
    new_rails[name] = rail

rail_type
        

{'CONV', 'FERRY', 'HSR', 'LINK', 'RR'}

In [3]:
import json
open("../data/_stations.json", "w", encoding='utf-8').write(json.dumps(
    json.loads(open("../data/stations.json", encoding='utf-8').read())
    , ensure_ascii=False, indent=4))

4089528

{'F', 货运
 'F2P1',
  None,
   'P', 客运
    'P2F1',
     'PF', 客运+货运
      'RR'}


{'',
 'ABZ+lr',
 'ABZlf',
 'ABZlg',
 'ABZlr',
 'ABZrf',
 'ABZrg',
 'BHF',  主站点
 'BST',  线路所
 'CONTf', 继续
 'CONTg', 继续
 'ENDEa', 继续
 'ENDEe', 继续
 'FLUG',  机场
 'GRZq',  关卡 过滤
 'HST',  小站点
 'KBHFa', 起点
 'KBHFe', 起点
 'KHSTa', 起点
 'KHSTe', 起点
 'KRZo',
 'STR',
 'STR+GRZq', 关卡 过滤
 'STRf',
 'STRg',
 'STRlf',
 'STRlg',
 'STRrf',
 'STRrg',
 'TRAJEKT', 轮渡 过滤
 'TUNNEL1', 隧道 过滤
 'WASSERq', 大桥 过滤
 'WBRÜCKE', 河 过滤
 'WBRüCKE;lZOLL', 河 过滤
 'WTUNNEL', 隧道 过滤
 'WTUNNEL1', 隧道 过滤
 'eBHF', 正常
 'eHST' 正常}